This is a notebook for caluclating the resistivity profile at SNMR sites using the AEM survey nearest point

Neil Symingotn

neil.symington@ga.gov.au


In [5]:
import pandas as pd
import numpy as np
import netCDF4
from hydrogeol_utils import spatial_functions
from geophys_utils import NetCDFPointUtils
import sqlalchemy as db
from sqlite3 import dbapi2 as sqlite
from sqlalchemy import create_engine, event
import os

In [34]:
# Connect to the SNMR database

DB_PATH = r"C:\Users\PCUser\Desktop\SSC_data\SNMR\SSC_SNMR.sqlite"
        
engine = db.create_engine('sqlite:///' + DB_PATH, module=sqlite)

@event.listens_for(engine, 'connect')
def connect(dbapi_connection, connection_rec):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("mod_spatialite")')


connection = engine.connect()

query = """

SELECT
              s.site_id,
              s.Field_ID,
              s.mid_X,
              s.mid_Y
              
FROM 

             sites as s

"""

df_sites = pd.read_sql_query(query, connection, index_col = 'site_id')


In [35]:
df_sites

,Field_ID,mid_X,mid_Y
site_id,,,
0,AS_GMR_15A,402460.3760,7360850.562
1,AS_GMR_23,440686.9021,7368656.659
2,AS_GMR_20,421235.9865,7364864.674
3,AS_GMR_22,429375.4174,7366308.744
4,AS_GMR_21,423033.3493,7368303.693
...,...,...,...
71,TT_GMR_15,372440.5256,7493864.659
72,TT_GMR_12,359844.8114,7525739.883
73,TT_GMR_06,340950.1837,7527435.463


In [36]:
coords = df_sites[['mid_X', 'mid_Y']]

In [37]:
# Bring in the LCI inversion

nc_inpath = r"C:\Users\PCUser\Desktop\AEM\LCI\SouthernStuart_WB_MGA53.nc"

d = netCDF4.Dataset(nc_inpath)

In [38]:
cond_point_utils = NetCDFPointUtils(d)

# Get the AEM utm coordinates

utm_wkt, aem_coords = cond_point_utils.utm_coords(cond_point_utils.xycoords)

In [39]:
# Extract the AEM conductivity using nearest neighbour
distances, indices = spatial_functions.nearest_neighbours(coords,
                                                          aem_coords,
                                                          points_required = 1,# return 10 closest points
                                                          max_distance = 10000.)

In [40]:
# Now we want to use these indices to extract the conductivity profile

conds = d['conductivity'][indices].data
res = 1./conds
depth_top = d['layer_top_depth'][indices].data

# Check all depths are the same

for i in range(depth_top.shape[1]):
    assert np.all(np.isclose(depth_top[:,i], depth_top[0,i]))


In [41]:
conds.shape[1]

30

In [42]:
for j in range(conds.shape[1]):
    df_sites['cond_l' + str(j+1)] = conds[:,j]
for j in range(conds.shape[1]):
    df_sites['layer_top_l' + str(j+1)] = depth_top[:,j]

In [43]:
df_sites['distance2fid'] = distances

df_sites.to_csv(r"C:\temp\SSC_SNMR_nearest_LCI_fid.csv", index = False)

In [33]:
# calculate thickness

thickness = spatial_functions.depth_to_thickness(depth_top[0])

In [66]:
# Now we will write the first 22 layers out in the required format
# which is :
#
#nlayers
#res(layer 1) thickness(layer 1)
#...
#res(layer 1) thickness(layer n)
# res bottom half space

nlayers = 22

outdir =  r"E:\SSC\NMR\SNMR\resisitivity_profiles"

for i, (index, row) in enumerate(df_sites.iterrows()):
    fname = os.path.join(outdir, row["Field_ID"] + '_res_profile.txt')
    print(fname)
    with open(fname, 'w') as f:
        f.write(str(nlayers) + '\n')
        for j in range(nlayers - 1):
            f.write(str(round(res[i,j],4)) + '\t')
            f.write(str(round(thickness[j],4)) + '\n')
        f.write(str(round(res[i,j+1],4)))

E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_02_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_07_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_08_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_09_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_26_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_30_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_31_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_32_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_35_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_36_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_37_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_38_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_39_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_40_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_41_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS

In [61]:
outdir = r"E:\SSC\NMR\SNMR\inversions"

for i, (index, row) in enumerate(df_sites.iterrows()):
    dirname = os.path.join(outdir, row["Field_ID"])
    os.mkdir(dirname)

In [59]:
path = r"E:\SSC\NMR\SNMR\Processed_data"
os.chdir(path)
import glob

files = [f for f in glob.glob(path + "**/*80ms*", recursive=True)]

In [60]:
files

[]